# Description

It combines all clustering solutions generated into a single consolidated solution using consensus clustering.

# Environment variables

In [1]:
from IPython.display import display

# set numpy n_jobs to 1, since I'll be using n_jobs later
NUMPY_N_JOBS = 1
display(NUMPY_N_JOBS)

1

In [2]:
%env MKL_NUM_THREADS=$NUMPY_N_JOBS
%env OPEN_BLAS_NUM_THREADS=$NUMPY_N_JOBS
%env NUMEXPR_NUM_THREADS=$NUMPY_N_JOBS
%env OMP_NUM_THREADS=$NUMPY_N_JOBS

env: MKL_NUM_THREADS=1
env: OPEN_BLAS_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: OMP_NUM_THREADS=1


# Modules loading

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path

import numpy as np
import pandas as pd

import conf

# Settings

In [5]:
RANDOM_GENERATOR = np.random.default_rng(12345)

## Ensemble size

In [6]:
EXPECTED_ENSEMBLE_SIZE = 295

MIN_ENSEMBLE_SIZE = 290
MAX_ENSEMBLE_SIZE = 300

## Consensus clustering

In [7]:
CLUSTERING_OPTIONS = {}

CLUSTERING_OPTIONS["K_MIN"] = 2
CLUSTERING_OPTIONS["K_MAX"] = 40

display(CLUSTERING_OPTIONS)

{'K_MIN': 2, 'K_MAX': 40}

In [8]:
# output dir for this notebook
RESULTS_DIR = Path(conf.RESULTS["CLUSTERING_DIR"], "consensus_clustering").resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering')

# Get ensemble

In [9]:
output_file = Path(RESULTS_DIR, "ensemble.npy").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/ensemble.npy')

In [10]:
full_ensemble = np.load(output_file)

In [11]:
display(full_ensemble.shape)

(4428, 3749)

# Get ensemble coassociation distance matrix

In [12]:
output_file = Path(RESULTS_DIR, "ensemble_coassoc_matrix.npy").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/ensemble_coassoc_matrix.npy')

In [13]:
ensemble_coassoc_matrix = np.load(output_file)

In [14]:
display(ensemble_coassoc_matrix.shape)

(3749, 3749)

In [15]:
display(ensemble_coassoc_matrix)

array([[0.        , 0.05004529, 0.04868659, ..., 0.62794384, 0.66445714,
        0.67006341],
       [0.05004529, 0.        , 0.02241848, ..., 0.62771739, 0.66514286,
        0.66779891],
       [0.04868659, 0.02241848, 0.        , ..., 0.62703804, 0.66285714,
        0.66530797],
       ...,
       [0.62794384, 0.62771739, 0.62703804, ..., 0.        , 0.59288808,
        0.66282746],
       [0.66445714, 0.66514286, 0.66285714, ..., 0.59288808, 0.        ,
        0.6127194 ],
       [0.67006341, 0.66779891, 0.66530797, ..., 0.66282746, 0.6127194 ,
        0.        ]])

# Consensus clustering

In [16]:
from concurrent.futures import ProcessPoolExecutor, as_completed

from tqdm import tqdm

from clustering.ensemble import (
    eac_single_coassoc_matrix,
    eac_complete_coassoc_matrix,
    eac_average_coassoc_matrix,
    run_method_and_compute_agreement,
)

In [17]:
all_consensus_methods = set(
    (
        eac_single_coassoc_matrix,
        eac_complete_coassoc_matrix,
        eac_average_coassoc_matrix,
    )
)
display(all_consensus_methods)

{<function clustering.ensemble.eac_average_coassoc_matrix(coassoc_matrix, k)>,
 <function clustering.ensemble.eac_complete_coassoc_matrix(coassoc_matrix, k)>,
 <function clustering.ensemble.eac_single_coassoc_matrix(coassoc_matrix, k)>}

In [18]:
consensus_results = []

with ProcessPoolExecutor(max_workers=conf.GENERAL["N_JOBS"]) as executor:
    tasks = {
        executor.submit(
            run_method_and_compute_agreement,
            m,
            ensemble_coassoc_matrix,
            full_ensemble,
            k,
        ): (m.__name__, k)
        for m in all_consensus_methods
        for k in range(CLUSTERING_OPTIONS["K_MIN"], CLUSTERING_OPTIONS["K_MAX"] + 1)
    }

    for future in tqdm(as_completed(tasks), total=len(tasks), disable=False, ncols=100):
        method_name, k = tasks[future]
        part, performance_values = future.result()

        method_results = {
            "method": method_name,
            "partition": part,
            "k": k,
        }
        method_results.update(performance_values)

        consensus_results.append(method_results)

  0%|                                                                       | 0/117 [00:00<?, ?it/s]

  1%|▌                                                              | 1/117 [00:23<45:43, 23.65s/it]

  2%|█                                                              | 2/117 [00:24<32:25, 16.92s/it]

  3%|█▌                                                             | 3/117 [00:26<23:25, 12.33s/it]

  3%|██▏                                                            | 4/117 [00:49<29:12, 15.51s/it]

...

 97%|███████████████████████████████████████████████████████████▍ | 114/117 [21:18<00:44, 14.72s/it]

 98%|███████████████████████████████████████████████████████████▉ | 115/117 [21:29<00:27, 13.51s/it]

 99%|████████████████████████████████████████████████████████████▍| 116/117 [21:47<00:14, 14.72s/it]

100%|█████████████████████████████████████████████████████████████| 117/117 [22:01<00:00, 14.68s/it]

100%|█████████████████████████████████████████████████████████████| 117/117 [22:01<00:00, 11.30s/it]

In [19]:
# TODO: check if each partition is really generating k clusters

In [20]:
consensus_results = pd.DataFrame(consensus_results)

In [21]:
display(consensus_results.shape)

(117, 12)

In [22]:
consensus_results.head()

,method,partition,k,ari_mean,ari_median,ari_std,ami_mean,ami_median,ami_std,nmi_mean,nmi_median,nmi_std
0,eac_average_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0.240317,0.037024,0.349342,0.249490,0.080186,0.324666,0.251982,0.082369,0.323813
1,eac_average_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,0.263578,0.054609,0.356479,0.284536,0.112219,0.323680,0.288405,0.116221,0.322205
2,eac_average_coassoc_matrix,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",4,0.287902,0.124714,0.331105,0.315595,0.187761,0.275006,0.320842,0.192473,0.273039
3,eac_average_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5,0.287923,0.124713,0.331126,0.318194,0.191098,0.276620,0.324061,0.195962,0.274261
4,eac_average_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",6,0.223957,0.147902,0.226714,0.299478,0.262243,0.142417,0.306541,0.268924,0.141986


## Save

In [23]:
output_file = Path(RESULTS_DIR, "consensus_clustering_runs.pkl").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/consensus_clustering_runs.pkl')

In [24]:
consensus_results.to_pickle(output_file)